In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import requests
from lxml.html import parse

from urllib.request import urlopen
from pandas.io.parsers import TextParser
import struct
print('Libraries imported.')

Libraries imported.


In [17]:
#select 1 bedroom and 1 bathroom
house_price = pd.read_csv('Toronto_apartment_rentals_2018.csv')
house_price = house_price.loc[house_price['Bedroom']==1]
house_price = house_price.loc[house_price['Bathroom']==1]
house_price.head()

,Bedroom,Bathroom,Den,Address,Lat,Long,Price
1,1,1.0,1,"361 Front St W, Toronto, ON M5V 3R5, Canada",43.643051,-79.391643,"$2,150.00"
2,1,1.0,0,"89 McGill Street, Toronto, ON, M5B 0B1",43.660605,-79.378635,"$1,950.00"
4,1,1.0,0,"80 St Patrick St, Toronto, ON M5T 2X6, Canada",43.652487,-79.389622,"$1,800.00"
5,1,1.0,0,"87 Jameson Avenue, Toronto, ON, M6K 2W8",43.634890,-79.434654,"$1,729.00"
7,1,1.0,0,"25 Telegram Mews, Toronto, ON, M5V 3Z2",43.640918,-79.393982,"$1,900.00"


In [18]:
# Select Latitude, Longitude and House price
house_price = house_price[['Price','Lat','Long']]
house_price.reset_index(inplace=True)
house_price.set_index('index',inplace=True)
house_price.head()

,Price,Lat,Long
index,,,
1,"$2,150.00",43.643051,-79.391643
2,"$1,950.00",43.660605,-79.378635
4,"$1,800.00",43.652487,-79.389622
5,"$1,729.00",43.634890,-79.434654
7,"$1,900.00",43.640918,-79.393982


In [20]:
house_price.to_csv("test.csv",index =False)
house_price =pd.read_csv("test.csv")
house_price.head()

,Price,Lat,Long
0,"$2,150.00",43.643051,-79.391643
1,"$1,950.00",43.660605,-79.378635
2,"$1,800.00",43.652487,-79.389622
3,"$1,729.00",43.634890,-79.434654
4,"$1,900.00",43.640918,-79.393982


In [21]:
#pre-process Price attribute
for index in range(len(house_price)):
    house_price['Price'][index] = house_price['Price'][index].replace("$", "")
    house_price['Price'][index] = house_price['Price'][index].replace(",", "")
    house_price['Price'][index] = pd.to_numeric(house_price['Price'][index], errors='coerce').astype(np.int64)

house_price.head()    

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Price,Lat,Long
0,2150,43.643051,-79.391643
1,1950,43.660605,-79.378635
2,1800,43.652487,-79.389622
3,1729,43.634890,-79.434654
4,1900,43.640918,-79.393982


In [23]:
#Use K-Means to cluster toronto house rental price


# set number of clusters
kclusters = 10
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(house_price)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([2, 8, 8, 3, 8, 8, 5, 3, 2, 5], dtype=int32)

In [24]:
#add cluster label to house_price data

house_price.insert(0, 'Cluster Labels', kmeans.labels_)

house_price.head() # check the last columns!


,Cluster Labels,Price,Lat,Long
0,2,2150,43.643051,-79.391643
1,8,1950,43.660605,-79.378635
2,8,1800,43.652487,-79.389622
3,3,1729,43.634890,-79.434654
4,8,1900,43.640918,-79.393982


In [28]:
house_price = house_price.rename(columns={"Lat":"Latitude","Long":"Longitude"})
house_price.head()

,Cluster Labels,Price,Latitude,Longitude
0,2,2150,43.643051,-79.391643
1,8,1950,43.660605,-79.378635
2,8,1800,43.652487,-79.389622
3,3,1729,43.634890,-79.434654
4,8,1900,43.640918,-79.393982


In [30]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent=None)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(house_price['Latitude'], house_price['Longitude'], house_price['Price'], house_price['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster)+'  with price '+ str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
#cluster statistic dataframe
price_result = pd.DataFrame(columns=('cluster id','size','average',))

for cluster in range(10):
    size = len(house_price.loc[house_price['Cluster Labels'] == cluster])
    average = house_price.loc[house_price['Cluster Labels'] == cluster]['Price'].mean()
#     print("Cluster "+str(cluster)+" is size of "+str(size)+" with average price: "+str(int(average)))
    row={'cluster id':int(cluster),'size':size,'average':int(average)}
    price_result.loc[cluster]=row
price_result = price_result.sort_values(['size'],ascending = [False])
price_result

,cluster id,size,average
2,2,172,2187
8,8,171,1905
5,5,129,2474
3,3,101,1649
6,6,59,1302
0,0,37,982
7,7,21,3000
9,9,11,518
4,4,8,3927
1,1,3,535000


In [49]:
#get ride of last five clusters as outliers

house_price_cleaned = pd.DataFrame(columns=('Cluster Labels','Price','Latitude','Longitude'))

count = 0 
for house in range(len(house_price)):
    if house_price['Cluster Labels'][house] != 0 and house_price['Cluster Labels'][house] != 1 and house_price['Cluster Labels'][house] != 4 and house_price['Cluster Labels'][house] != 7 and house_price['Cluster Labels'][house] != 9:
#         print(house_price['Cluster Labels'][house])
        series = pd.Series({"Cluster Labels":house_price["Cluster Labels"][house],"Price":house_price["Price"][house],"Latitude":house_price["Latitude"][house],"Longitude":house_price["Longitude"][house]})
        house_price_cleaned = house_price_cleaned.append(series,ignore_index=True)
        count +=1
print(count)

#reorginise label from 0-4

labels = house_price_cleaned['Cluster Labels']
for i in range(len(house_price_cleaned)):
    if labels[i] == 5.0:
        labels[i] = 0
    if labels[i] == 6:
        labels[i] = 1
    if  labels[i] == 8:
        labels[i] = 4
        
house_price_cleaned.head()


632


,Cluster Labels,Price,Latitude,Longitude
0,2.0,2150.0,43.643051,-79.391643
1,4.0,1950.0,43.660605,-79.378635
2,4.0,1800.0,43.652487,-79.389622
3,3.0,1729.0,43.634890,-79.434654
4,4.0,1900.0,43.640918,-79.393982


In [54]:
#Draw the map

address = 'Toronto, CA'

geolocator = Nominatim(user_agent=None)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(house_price_cleaned['Latitude'], house_price_cleaned['Longitude'], house_price_cleaned['Price'], house_price_cleaned['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster)+'  with price '+ str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.
